In [ ]:
import math

def find_linear_approximation_errors(f, c, E, delta_x=1e-8, max_iter=10000, step_size=1e-4):
    """
    Finds two closest distinct values x1 and x2 around c where the absolute error
    between f(x) and its linear approximation L(x) is exactly E.

    Parameters:
    f (function): The function to approximate.
    c (float): The point around which to linearize.
    E (float): The desired absolute error.
    delta_x (float): The step size for central difference method.
    max_iter (int): Maximum number of iterations to search for x1 and x2.
    step_size (float): Initial step size for searching x1 and x2.

    Returns:
    tuple: (x1, x2) if found, or an error message if not found.
    """

    # Approximate the derivative using central difference method
    def approximate_derivative(f, c, delta_x):
        return (f(c + delta_x) - f(c - delta_x)) / (2 * delta_x)

    f_c = f(c)
    f_prime_c = approximate_derivative(f, c, delta_x)

    # Linear approximation function
    def L(x):
        return f_c + f_prime_c * (x - c)

    # Function to find root: |f(x) - L(x)| - E = 0
    def error_diff(x):
        return abs(f(x) - L(x)) - E

    # Binary search to find where error_diff crosses zero
    def find_root(start, direction, max_iter=max_iter, initial_step=step_size):
        x = start
        step = initial_step
        iterations = 0

        # First find a bracket where error_diff changes sign
        while iterations < max_iter:
            iterations += 1
            current_error = error_diff(x)
            new_x = x + direction * step
            new_error = error_diff(new_x)

            if current_error * new_error <= 0:  # Sign change detected
                break
            x = new_x

            # Adjust step size if we're not making progress
            if abs(new_error) > abs(current_error):
                step *= 0.5
        else:
            return None  # No root found

        # Now refine the root using binary search
        left, right = (x, new_x) if direction > 0 else (new_x, x)
        for _ in range(max_iter):
            mid = (left + right) / 2
            mid_error = error_diff(mid)

            if abs(mid_error) < 1e-12:  # Close enough to zero
                return mid
            elif mid_error * error_diff(left) < 0:
                right = mid
            else:
                left = mid

        return (left + right) / 2

    # Find x1 (left of c)
    x1 = find_root(c, -1)
    # Find x2 (right of c)
    x2 = find_root(c, 1)

    if x1 is not None and x2 is not None and x1 != x2:
        return (x1, x2)
    else:
        return "No suitable x1 and x2 found within the search range."

# Test cases
if __name__ == "__main__":
    # Test case (i): f(x) = x^2, c=1, E=0.1
    def f1(x): return x**2
    result1 = find_linear_approximation_errors(f1, 1, 0.1)
    print(f"Test case (i) x^2 at c=1, E=0.1: {result1}")

    # Test case (ii): f(x) = sin(x), c=pi/4, E=0.05
    def f2(x): return math.sin(x)
    result2 = find_linear_approximation_errors(f2, math.pi/4, 0.05)
    print(f"Test case (ii) sin(x) at c=pi/4, E=0.05: {result2}")

    # Test case (iii): f(x) = e^x, c=0, E=0.01
    def f3(x): return math.exp(x)
    result3 = find_linear_approximation_errors(f3, 0, 0.01)
    print(f"Test case (iii) e^x at c=0, E=0.01: {result3}")